In [3]:
import pandas as pd
import torch

In [4]:
import numpy as np

In [52]:
splits=np.load("data/splits.npz")
splits=dict(splits)

In [53]:
splits["idx_val"][19]

1043

In [3]:
df=pd.read_csv("data/tox21/raw/tox21.csv")
smiles_list=list(df["smiles"])
target = df[['NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD',
           'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53']]

target = target.replace(0, -1)  # convert 0 to -1
target = target.fillna(0)  

target = torch.tensor(target.values,dtype=torch.float)

In [4]:
target.shape

torch.Size([7831, 12])

In [7]:
target[1].unsqueeze(0).shape

torch.Size([1, 12])

In [1]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdchem import HybridizationType
from rdkit.Chem.rdchem import BondType as BT
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
from tqdm import tqdm
from rdkit.Chem import Draw
import py3Dmol
from rdkit.Chem.Draw import IPythonConsole


In [8]:
df=pd.read_csv("data/bbbp/raw/bbbp.csv")
smiles_list=list(df["smiles"])

In [87]:
def get_MMFF_mol(self,mol,numConfs=1):
        try:
            new_mol = Chem.AddHs(mol)
            res = AllChem.EmbedMultipleConfs(new_mol, numConfs=numConfs,maxAttempts=1000)
            res = AllChem.MMFFOptimizeMoleculeConfs(new_mol)
        except:
            return None
        return new_mol

In [47]:
from torchmdnet.datasets import BBBP
from torch.utils.data import Subset

In [50]:
dataset = BBBP("data/bbbp", "homo",num_confs=1)
subset_dataset=Subset(dataset, splits["idx_test"])


In [51]:
subset_dataset

In [30]:
dataset[[10,12]]

BBBP(2)

In [5]:
data=torch.load("data/bbbp/processed/data_v3.pt")

In [66]:
index=19
data[0]["name"][index]

'0-Nc1nnc(c(N)n1)c2cccc(Cl)c2Cl'

In [67]:
smiles=data[0]["name"][index][2:]
mol = AllChem.MolFromSmiles(smiles)
#display(Draw.MolToImage(mol))
mol = Chem.AddHs(mol)
res = AllChem.EmbedMultipleConfs(mol, numConfs=1)
res = AllChem.MMFFOptimizeMoleculeConfs(mol)
#Draw.IPythonConsole.drawMol3D(mol)
print(torch.tensor(mol.GetConformer().GetPositions()).shape)
print(torch.unique(torch.tensor(mol.GetConformer().GetPositions()), dim=0).shape)

torch.Size([23, 3])
torch.Size([23, 3])


In [ ]:
bad mols = [[ 19]
 [ 59]
 [ 71]
 [288]
 [342]
 [390]]

In [10]:
symbol_list=[]
for i, smile in enumerate(tqdm(df["smiles"])):
    mol = AllChem.MolFromSmiles(smile)
    if mol is None:
        continue
    for atom in mol.GetAtoms():
        symb=atom.GetSymbol()
        if symb not in symbol_list:
            symbol_list.append(symb)

  0%|          | 0/2050 [00:00<?, ?it/s]

100%|██████████| 2050/2050 [00:00<00:00, 2272.44it/s]


In [11]:
strm=""
for i,symb in enumerate(symbol_list):
    st="\'"+symb+"\'"+":"+str(i)+","
    strm+=st

In [12]:
strm

"'Cl':0,'C':1,'N':2,'O':3,'F':4,'S':5,'Br':6,'I':7,'H':8,'Na':9,'P':10,'Ca':11,'B':12,"